<a href="https://colab.research.google.com/github/MarcosArambasic/LIMA---Marcos-Arambasic/blob/main/runningMPI_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install openmpi-bin openmpi-common libopenmpi-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopenmpi-dev is already the newest version (4.1.2-2ubuntu1).
openmpi-bin is already the newest version (4.1.2-2ubuntu1).
openmpi-bin set to manually installed.
openmpi-common is already the newest version (4.1.2-2ubuntu1).
openmpi-common set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [3]:
%%writefile teste-mpi.c
#include "mpi.h"
#include <stdio.h>
#include <math.h>
#include <unistd.h>
#include <stdlib.h>

#define MAXSIZE 1000
int main(int argc, char **argv)
{
	int myid, numprocs;
	int data[MAXSIZE], i, x, low, high, myresult=0, result=0;
	FILE *fp;
	MPI_Init(&argc,&argv);
	MPI_Comm_size(MPI_COMM_WORLD,&numprocs);
	MPI_Comm_rank(MPI_COMM_WORLD,&myid);
	if (myid == 0) {  /* Open input file and initialize data */
		if ((fp = fopen("rand.txt","r")) == NULL) {
			printf("Can't open the input file: rand.txt\n\n");
			exit(1);
		}
		for(i = 0; i < MAXSIZE; i++) fscanf(fp,"%d\n", &data[i]);
	}
	MPI_Bcast(data, MAXSIZE, MPI_INT, 0, MPI_COMM_WORLD); /* broadcast data */
	x = MAXSIZE/numprocs; /* Add my portion Of data */
	low = myid * x;
	high = low + x;
	for(i = low; i < high; i++)
		myresult += data[i];
	char hostname[30];
	gethostname(hostname, 30);
	printf("I got %d from %d - %s\n", myresult, myid, hostname); /* Compute global sum */
	MPI_Reduce(&myresult, &result, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);
	if (myid == 0) printf("The sum is %d.\n", result);
	MPI_Finalize();
	return 0;
}



Overwriting teste-mpi.c


In [4]:
!mpicc -o teste-mpi teste-mpi.c

In [9]:
!mpiexec  --allow-run-as-root -n 4 ./teste-mpi

--------------------------------------------------------------------------
There are not enough slots available in the system to satisfy the 4
slots that were requested by the application:

  ./teste-mpi

Either request fewer slots for your application, or make more slots
available for use.

A "slot" is the Open MPI term for an allocatable unit where we can
launch a process.  The number of slots available are defined by the
environment in which Open MPI processes are run:

  1. Hostfile, via "slots=N" clauses (N defaults to number of
     processor cores if not provided)
  2. The --host command line parameter, via a ":N" suffix on the
     hostname (N defaults to 1 if not provided)
  3. Resource manager (e.g., SLURM, PBS/Torque, LSF, etc.)
  4. If none of a hostfile, the --host command line parameter, or an
     RM is present, Open MPI defaults to the number of processor cores

In all the above cases, if you want Open MPI to default to the number
of hardware threads instead of the numb